# Covid vs India 
> One of the biggest question I have been wondering is , with every adult allowed now to register, Where are vaccines slots available in india ? How can I make this decision programmitcally because trying to find information on government website is rather "sisyphus task".
So with rather self involed goal in mind, We would using publicly avaialbe data try to answer following questions
- toc: false
- branch: master
- badges: false
- comments: true
- categories: [dfs, jupyter, python]
- image: images/statistics.png
- hide: false

1. %of free vs paid ,( piechart ?)
2. how many centers per state and who has best ratio against population? ( chart of top 5 and show it on map )
3. at current allocation how many days will it take to vaccinate 18+?
2. what's ground reality for Key states like Delhi, Maharastra , Tamil Nadu
2. What's ground reality for populus states like UP, MP, 
3. How does it fare around 
2. Latest vaccine slots per district and State in India
2. Top 5 States Running behind on schedule
3. Top 5 States Running ahead of everyone else.
4. When can I latest find slots in Delhi, Bombay, Chennai and Bangalore for people between 18-45



Thanks to https://github.com/bhattbhavesh91/cowin-vaccination-slot-availability/blob/main/cowin-api-availability.ipynb for doing actual work, I picked up loads of stuff from there.


In [36]:
!pip install --user install requests
!pip install --user altair
!pip install --user install pandas
!pip install --user install geopandas
#hide

You should consider upgrading via the '/Users/saimeerashanawas/.pyenv/versions/3.9.4/bin/python3.9 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/saimeerashanawas/.pyenv/versions/3.9.4/bin/python3.9 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/saimeerashanawas/.pyenv/versions/3.9.4/bin/python3.9 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/saimeerashanawas/.pyenv/versions/3.9.4/bin/python3.9 -m pip install --upgrade pip' command.


Get All libraries in place

In [70]:
import geopandas as gpd
import pandas as pd
import altair as alt
import requests
import json
from collections import defaultdict
from dataclasses import dataclass, asdict, field
from datetime import datetime, timedelta
from typing import List
import uuid

Lets make a data class to store our geographical and vaccination data

In [38]:
@dataclass
class District:
    district_id:int = None
    district_name:str = None
    state_id:int = None
    district_created_at: datetime = field(default_factory=datetime.now)


@dataclass
class Session:
    session_uuid:str = None
    date:datetime = None
    query_date:datetime= None
    available_capacity:int = None
    min_age_limit:int = None
    vaccine:str = None
    center_id:str = None
    district_id:str = None
    session_created_at: datetime = field(default_factory=datetime.now)
        
@dataclass
class Center:
    center_uuid:str = None
    center_id:int = None
    center_name:str = None
    state_name:str = None
    district_name:str = None
    block_name:str = None
    pincode:str = None
    lat:int = None
    lng:int = None
    from_hour:datetime = None
    to_hour:datetime = None
    fee_type:str = None
    district_id:str = None
    center_created_at: datetime = field(default_factory=datetime.now)

@dataclass
class NoSlotAvailable:
    district_id:str = None
    date:datetime = None
    no_slot_available_created_at: datetime = field(default_factory=datetime.now)

Lets call the API to get the populate geographical data

In [39]:
MOZILLA_HEADER = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'
GET_DISTRICT_DATA_API_URL = "https://cdn-api.co-vin.in/api/v2/admin/location/districts/{}"
GET_APOINTMENT_DATA_API_URL = "https://cdn-api.co-vin.in/api/v2/appointment/sessions/public/calendarByDistrict?district_id={}&date={}"

DISTRICTS = defaultdict(District)
CENTERS = defaultdict(Center)
SESSIONS = defaultdict(Session)
NO_SLOT_AVAILABLE = []

MAX_NUMBER_OF_STATES = 40

for state_code in range(1, MAX_NUMBER_OF_STATES):
    headers = {'User-Agent': MOZILLA_HEADER}
    response = requests.get(GET_DISTRICT_DATA_API_URL.format(state_code), headers=headers)
    districts_data = json.loads(response.content)
    for district in districts_data['districts']:
        district_name = district['district_name']
        district_id = district['district_id']
        district = District(district_name = district_name, district_id = district_id , state_id=state_code) 
        DISTRICTS[district_id] = district

In [40]:
DISTRICT_DF = pd.DataFrame.from_dict([asdict(district) for district in DISTRICTS.values()])

Lets now call actual api to get slots 

In [41]:
#hide_output
MAX_DAYS = 7

#This step takes loads of time
def get_days_in_future_from_today():
    base = datetime.today()
    date_list = [base + timedelta(days=x) for x in range(MAX_DAYS)]
    return [x.strftime("%d-%m-%Y") for x in date_list]

for district_id in DISTRICTS.keys():
    print("District {}".format(district_id))
    for slot_date in get_days_in_future_from_today():
        URL = GET_APOINTMENT_DATA_API_URL.format(district_id, slot_date)
        response = requests.get(URL)
        if response.ok:
            resp_json = response.json()
            if resp_json["centers"]:
                for center in resp_json["centers"]:
                    center_uuid = str(uuid.uuid4())
                    center_id = center["center_id"]
                    center_name = center["name"]
                    CENTERS[center_uuid] = Center(center_uuid=center_uuid,
                                                center_id=center_id,
                                                center_name=center_name,
                                                lat=center["lat"],
                                                lng=center["long"],
                                                from_hour=center["from"],
                                                to_hour=center["to"],
                                                district_id=district_id,
                                                state_name=center["state_name"],
                                                district_name=center["district_name"],
                                                block_name=center["block_name"],
                                                pincode=center["pincode"],
                                                fee_type=center["fee_type"])
                    for session in center["sessions"]:
                        session_id = session["session_id"]
                        SESSIONS[session_id] = Session(session_uuid=session_id,
                                                       date=session["date"],
                                                       query_date=slot_date,
                                                       available_capacity=session["available_capacity"],
                                                       min_age_limit=session["min_age_limit"],
                                                       vaccine=session["vaccine"],
                                                       district_id=district_id,
                                                       center_id=center_id)
            else:
                NO_SLOT_AVAILABLE.append(NoSlotAvailable(district_id=district_id, date=slot_date))
#                 print("No slot on {} in district {}".format(slot_date, district_id))

District 3
District 1
District 2
District 9
District 10
District 11
District 5
District 4
District 7
District 12
District 13
District 14
District 8
District 15
District 16
District 6
District 22
District 20
District 25
District 23
District 42
District 17
District 24
District 27
District 21
District 33
District 29
District 40
District 31
District 18
District 32
District 36
District 19
District 39
District 35
District 37
District 30
District 26
District 34
District 41
District 28
District 38
District 46
District 47
District 765
District 57
District 66
District 766
District 58
District 48
District 62
District 59
District 43
District 67
District 60
District 53
District 68
District 764
District 54
District 49
District 50
District 51
District 69
District 61
District 63
District 767
District 55
District 56
District 52
District 44
District 64
District 768
District 45
District 65
District 769
District 74
District 78
District 77
District 83
District 98
District 82
District 99
District 100
Distri

District 625
District 626
District 627
District 628
District 646
District 629
District 630
District 631
District 632
District 633
District 634
District 635
District 636
District 637
District 638
District 687
District 639
District 640
District 641
District 642
District 643
District 644
District 645
District 647
District 648
District 649
District 650
District 651
District 652
District 653
District 654
District 655
District 656
District 657
District 658
District 659
District 660
District 661
District 662
District 663
District 664
District 665
District 666
District 667
District 668
District 669
District 670
District 671
District 672
District 673
District 674
District 675
District 676
District 677
District 678
District 679
District 680
District 682
District 624
District 681
District 683
District 684
District 685
District 686
District 688
District 689
District 690
District 691
District 692
District 693
District 694
District 695
District 696
District 704
District 707
District 699
District 708

In [42]:
CENTER_DF = pd.DataFrame.from_dict([asdict(district) for district in CENTERS.values()])
SESSION_DF = pd.DataFrame.from_dict([asdict(session) for session in SESSIONS.values()])
NO_SLOT_AVAILABLE_DF = pd.DataFrame.from_dict([asdict(no_slot_available) for no_slot_available in NO_SLOT_AVAILABLE])

In [43]:
NO_SLOT_AVAILABLE_DF.head()

,district_id,date,no_slot_available_created_at
0,5,04-05-2021,2021-05-03 16:22:33.578494
1,5,05-05-2021,2021-05-03 16:22:33.744831
2,5,06-05-2021,2021-05-03 16:22:33.859553
3,5,07-05-2021,2021-05-03 16:22:33.970119
4,5,08-05-2021,2021-05-03 16:22:34.321653


In [44]:
SESSION_DF.head()

,session_uuid,date,query_date,available_capacity,min_age_limit,vaccine,center_id,district_id,session_created_at
0,207fd35f-888d-460d-8304-f0b99f9a13a0,03-05-2021,03-05-2021,49.0,45,,570779,3,2021-05-03 16:22:25.027502
1,88373311-b0da-40f6-bbea-ee029f86eb1c,04-05-2021,04-05-2021,50.0,45,,570779,3,2021-05-03 16:22:25.264174
2,8ef1ec99-7318-4cb6-bc66-9ed3675f9a48,05-05-2021,05-05-2021,50.0,45,,570779,3,2021-05-03 16:22:25.412119
3,bd311aec-c4cf-45f3-8f06-2dbc7b634de4,06-05-2021,06-05-2021,50.0,45,,570779,3,2021-05-03 16:22:25.628695
4,f6022b09-7079-43fe-a48c-dc82e04a0058,08-05-2021,08-05-2021,49.0,45,,570779,3,2021-05-03 16:22:25.982494


In [45]:
CENTER_DF.head()

,center_uuid,center_id,center_name,state_name,district_name,block_name,pincode,lat,lng,from_hour,to_hour,fee_type,district_id,center_created_at
0,68709bda-d461-4295-b453-0446e3c3836d,570779,BJR Hospital,Andaman and Nicobar Islands,Nicobar,Car Nicobar,744301,0,0,09:00:00,17:00:00,Free,3,2021-05-03 16:22:25.027457
1,39004cf0-9221-4b91-a3ed-efbd4d129d25,639974,MI Room DHQ 10,Andaman and Nicobar Islands,Nicobar,Campbell Bay,744302,7,93,09:00:00,18:00:00,Free,3,2021-05-03 16:22:25.027546
2,4b6ad957-cc5c-489b-8b40-028512b83dbf,552109,Campbellbay PHC,Andaman and Nicobar Islands,Nicobar,Campbell Bay,744302,7,93,09:00:00,17:00:00,Free,3,2021-05-03 16:22:25.027647
3,18036b66-4a41-4424-b5e5-672f5722ce6f,552108,Nancowry CHC,Andaman and Nicobar Islands,Nicobar,Nancowry,744303,7,93,09:00:00,17:00:00,Free,3,2021-05-03 16:22:25.027704
4,e44f0c92-4368-4a4e-b0de-020095e84dfb,639986,SMC 37 WING,Andaman and Nicobar Islands,Nicobar,Car Nicobar,744301,9,92,09:00:00,18:00:00,Free,3,2021-05-03 16:22:25.027753


In [46]:
DISTRICT_DF.head()

,district_id,district_name,state_id,district_created_at
0,3,Nicobar,1,2021-05-03 16:22:15.927874
1,1,North and Middle Andaman,1,2021-05-03 16:22:15.927885
2,2,South Andaman,1,2021-05-03 16:22:15.927887
3,9,Anantapur,2,2021-05-03 16:22:17.377390
4,10,Chittoor,2,2021-05-03 16:22:17.377401


In [47]:
session_center_merged_df = pd.merge(SESSION_DF, CENTER_DF, on="center_id")
session_center_district_df = pd.merge(session_center_merged_df, DISTRICT_DF, left_on='district_id_x', right_on="district_id")

In [48]:
session_center_district_df.to_csv("vaccination_slot_data.csv")
session_center_district_df

,session_uuid,date,query_date,available_capacity,min_age_limit,vaccine,center_id,district_id_x,session_created_at,center_uuid,...,lng,from_hour,to_hour,fee_type,district_id_y,center_created_at,district_id,district_name_y,state_id,district_created_at
0,207fd35f-888d-460d-8304-f0b99f9a13a0,03-05-2021,03-05-2021,49.0,45,,570779,3,2021-05-03 16:22:25.027502,68709bda-d461-4295-b453-0446e3c3836d,...,0,09:00:00,17:00:00,Free,3,2021-05-03 16:22:25.027457,3,Nicobar,1,2021-05-03 16:22:15.927874
1,207fd35f-888d-460d-8304-f0b99f9a13a0,03-05-2021,03-05-2021,49.0,45,,570779,3,2021-05-03 16:22:25.027502,60d78a27-077d-4d9e-b3a7-f0d78907f375,...,0,09:00:00,17:00:00,Free,3,2021-05-03 16:22:25.264162,3,Nicobar,1,2021-05-03 16:22:15.927874
2,207fd35f-888d-460d-8304-f0b99f9a13a0,03-05-2021,03-05-2021,49.0,45,,570779,3,2021-05-03 16:22:25.027502,87a707cc-6929-4ff3-94cf-e0c7d5dfe750,...,0,09:00:00,17:00:00,Free,3,2021-05-03 16:22:25.412101,3,Nicobar,1,2021-05-03 16:22:15.927874
3,207fd35f-888d-460d-8304-f0b99f9a13a0,03-05-2021,03-05-2021,49.0,45,,570779,3,2021-05-03 16:22:25.027502,887d3e0a-fd2a-4f69-9af7-9e8ce874d155,...,0,09:00:00,17:00:00,Free,3,2021-05-03 16:22:25.628681,3,Nicobar,1,2021-05-03 16:22:15.927874
4,207fd35f-888d-460d-8304-f0b99f9a13a0,03-05-2021,03-05-2021,49.0,45,,570779,3,2021-05-03 16:22:25.027502,f4d8eb46-0c16-4f88-8afa-6bc40c91b2e0,...,0,09:00:00,17:00:00,Free,3,2021-05-03 16:22:25.781478,3,Nicobar,1,2021-05-03 16:22:15.927874
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1251756,aa1390fe-2b5f-46c3-aab1-8807035a210b,04-05-2021,04-05-2021,10.0,45,,343857,139,2021-05-03 16:44:00.778635,ee1be1a2-ff91-4362-9cd8-68aa7492545c,...,70,09:00:00,17:00:00,Free,139,2021-05-03 16:44:00.778626,139,Diu,37,2021-05-03 16:22:24.553471
1251757,aa1390fe-2b5f-46c3-aab1-8807035a210b,04-05-2021,04-05-2021,10.0,45,,343857,139,2021-05-03 16:44:00.778635,c31e86c7-cb3f-4be5-9fcd-1cb1f3bacd3a,...,70,09:00:00,17:00:00,Free,139,2021-05-03 16:44:00.927269,139,Diu,37,2021-05-03 16:22:24.553471
1251758,56c835d4-b1be-4df7-97de-20b88f3d61ee,05-05-2021,05-05-2021,10.0,45,,343857,139,2021-05-03 16:44:00.927278,9143e567-5417-4623-abc1-ce6235b5f905,...,70,09:00:00,17:00:00,Free,139,2021-05-03 16:44:00.663340,139,Diu,37,2021-05-03 16:22:24.553471
1251759,56c835d4-b1be-4df7-97de-20b88f3d61ee,05-05-2021,05-05-2021,10.0,45,,343857,139,2021-05-03 16:44:00.927278,ee1be1a2-ff91-4362-9cd8-68aa7492545c,...,70,09:00:00,17:00:00,Free,139,2021-05-03 16:44:00.778626,139,Diu,37,2021-05-03 16:22:24.553471


In [49]:
# Top 5 States Running ahead of everyone else.
session_center_district_df[['date', 'state_name']].groupby('state_name').apply(lambda x : x.sort_values(by = 'date', ascending = True).head(3).reset_index(drop = True))

date                   state_name
state_name                                                            
Andaman and Nicobar Islands 0  03-05-2021  Andaman and Nicobar Islands
                            1  03-05-2021  Andaman and Nicobar Islands
                            2  03-05-2021  Andaman and Nicobar Islands
Andhra Pradesh              0  03-05-2021               Andhra Pradesh
                            1  03-05-2021               Andhra Pradesh
...                                   ...                          ...
Uttarakhand                 1  03-05-2021                  Uttarakhand
                            2  03-05-2021                  Uttarakhand
West Bengal                 0  03-05-2021                  West Bengal
                            1  03-05-2021                  West Bengal
                            2  03-05-2021                  West Bengal

[111 rows x 2 columns]

In [50]:
#hide
session_center_district_df\
    [['date', 'state_name']]\
    .groupby('state_name')\
    .apply(lambda x : x.sort_values(by = 'date', ascending = True)\
    .head(1)\
    .reset_index(drop = True)).head(40)

,,date,state_name
state_name,,,
Andaman and Nicobar Islands,0,03-05-2021,Andaman and Nicobar Islands
Andhra Pradesh,0,03-05-2021,Andhra Pradesh
Arunachal Pradesh,0,03-05-2021,Arunachal Pradesh
Assam,0,03-05-2021,Assam
Bihar,0,03-05-2021,Bihar
Chandigarh,0,03-05-2021,Chandigarh
Chhattisgarh,0,03-05-2021,Chhattisgarh
Dadra and Nagar Haveli,0,03-05-2021,Dadra and Nagar Haveli
Daman and Diu,0,03-05-2021,Daman and Diu


In [51]:
#hide
session_center_district_df\
    [session_center_district_df["min_age_limit"]<45]\
    [session_center_district_df["fee_type"]=="Free"]\
    [['date', 'state_name']]\
    .groupby('state_name')\
    .apply(lambda x : x.sort_values(by = 'date', ascending = True)\
    .head(1)\
    .reset_index(drop = True)).head(40)

<ipython-input-51-2df4eb2f42ac>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  session_center_district_df\


,,date,state_name
state_name,,,
Chhattisgarh,0,03-05-2021,Chhattisgarh
Delhi,0,03-05-2021,Delhi
Gujarat,0,03-05-2021,Gujarat
Haryana,0,03-05-2021,Haryana
Himachal Pradesh,0,04-05-2021,Himachal Pradesh
Jammu and Kashmir,0,03-05-2021,Jammu and Kashmir
Jharkhand,0,03-05-2021,Jharkhand
Karnataka,0,04-05-2021,Karnataka
Ladakh,0,03-05-2021,Ladakh


In [52]:
#hide
session_center_district_df.drop_duplicates(["date","state_id"]).groupby("state_id").apply(lambda x : x.sort_values(by = 'date', ascending = True).head(1).reset_index(drop = True)).head(40)

,,session_uuid,date,query_date,available_capacity,min_age_limit,vaccine,center_id,district_id_x,session_created_at,center_uuid,...,lng,from_hour,to_hour,fee_type,district_id_y,center_created_at,district_id,district_name_y,state_id,district_created_at
state_id,,,,,,,,,,,,,,,,,,,,,,
1,0,207fd35f-888d-460d-8304-f0b99f9a13a0,03-05-2021,03-05-2021,49.0,45,,570779,3,2021-05-03 16:22:25.027502,68709bda-d461-4295-b453-0446e3c3836d,...,0,09:00:00,17:00:00,Free,3,2021-05-03 16:22:25.027457,3,Nicobar,1,2021-05-03 16:22:15.927874
2,0,6fd27b67-b4fc-4a75-bb16-712e8b6416e9,03-05-2021,03-05-2021,27.0,45,,342878,9,2021-05-03 16:22:28.261063,3ceb68f4-bec2-49d3-94fc-7a478923d023,...,78,09:00:00,18:00:00,Free,9,2021-05-03 16:22:28.261050,9,Anantapur,2,2021-05-03 16:22:17.377390
3,0,71c332c3-e6f2-4591-a926-6bc847ea5980,03-05-2021,03-05-2021,37.0,45,,105352,20,2021-05-03 16:23:06.035292,e985e198-2938-4a40-865b-2a1d56193f78,...,96,09:00:00,17:00:00,Free,20,2021-05-03 16:23:06.035276,20,Changlang,3,2021-05-03 16:22:17.518601
4,0,983c3c82-d799-4dde-8141-f532ca221ad0,03-05-2021,03-05-2021,49.0,45,,571688,46,2021-05-03 16:23:45.682588,510efe9e-60b0-4dd1-a559-4fb3ea73c7dc,...,91,09:00:00,18:00:00,Free,46,2021-05-03 16:23:45.682566,46,Baksa,4,2021-05-03 16:22:17.983963
5,0,ecf2e2b8-56e7-48ee-b6b3-a515c51f496b,03-05-2021,03-05-2021,45.0,45,,615271,74,2021-05-03 16:24:31.799327,40973eaa-c6c1-4ed9-b869-923ed73da16f,...,87,09:00:00,18:00:00,Free,74,2021-05-03 16:24:31.798813,74,Araria,5,2021-05-03 16:22:18.107617
6,0,74ea8480-e12e-46d7-9c90-4aa95249c987,03-05-2021,03-05-2021,92.0,45,,199583,108,2021-05-03 16:25:45.918865,42d58fa3-879b-4689-bdd2-9116d7e853d7,...,76,09:00:00,15:00:00,Free,108,2021-05-03 16:25:45.918205,108,Chandigarh,6,2021-05-03 16:22:18.245516
7,0,009abc47-4c82-49cf-814c-cb6f62760f8b,03-05-2021,03-05-2021,50.0,45,,647454,110,2021-05-03 16:25:47.259734,b7cc1ad4-8df6-4e6e-922f-d32a81228cb4,...,81,09:00:00,18:00:00,Free,110,2021-05-03 16:25:47.259720,110,Balod,7,2021-05-03 16:22:18.387000
8,0,ac0ac6ca-6059-48ed-8f15-b8cecddeb72b,03-05-2021,03-05-2021,0.0,45,,351229,137,2021-05-03 16:27:06.900124,ff4cfbe4-97c3-4b7a-9c1f-058d94560a54,...,73,09:00:00,17:00:00,Free,137,2021-05-03 16:27:06.900109,137,Dadra and Nagar Haveli,8,2021-05-03 16:22:18.510570
9,0,bdcae387-bc24-45e2-a5dc-0f1378b8ede1,03-05-2021,03-05-2021,21.0,45,,4689,141,2021-05-03 16:27:09.200471,4dc1ec19-f6fe-4a4b-824e-7f7ed6ef65ea,...,77,09:00:00,17:00:00,Free,141,2021-05-03 16:27:09.200456,141,Central Delhi,9,2021-05-03 16:22:18.648308


In [91]:
#Lets Find out how many 18-45 range people would be able to get the vaccine for free ?
sessions_for_below_45 = session_center_district_df\
[session_center_district_df["min_age_limit"] <45]

sessions_for_below_45_free_fee_type_count = len(sessions_for_below_45[sessions_for_below_45["fee_type"]=="Free"])
sessions_for_below_45_not_free_fee_type_count = len(sessions_for_below_45) - sessions_for_below_45_free_fee_type_count
sessions_for_below_45_not_free_fee_type_count, sessions_for_below_45_free_fee_type_count

source = pd.DataFrame({
    "y":["Free Vaccines Centers","Paid Vaccines Centers"],
    "x":[sessions_for_below_45_not_free_fee_type_count,sessions_for_below_45_free_fee_type_count]
})

bars = alt.Chart(source).transform_joinaggregate(
    TotalX='sum(x)',
).transform_calculate(
    PercentOfTotal="datum.x / datum.TotalX"
).mark_bar(
    cornerRadiusTopLeft=3,
    cornerRadiusTopRight=3
).encode(
    alt.X('PercentOfTotal:Q', axis=alt.Axis(format='.0%')),
    y='y',
)

text = bars.mark_text(
    align='left',
    baseline='middle',
    dx=3  # Nudges text to right so it doesn't appear on top of the bar
).encode(
    text='x'
)

(bars+text).properties(height=200)

alt.LayerChart(...)